In [2]:
import random
import numpy as np
from PIL import Image
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Compose, Normalize
from image_classification import generate_dataset
from helpers import index_splitter, make_balanced_sampler
from v1 import StepByStep


d:\software\miniconda\envs\mlenv\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
single = np.array(
  [[[[5, 0, 8, 7, 8, 1],
  [1, 9, 5, 0, 7, 7],
  [6, 0, 2, 4, 6, 6],
  [9, 7, 6, 6, 8, 4],
  [8, 3, 8, 5, 1, 3],
  [7, 2, 7, 0, 1, 0]]]]
)
single.shape

(1, 1, 6, 6)

In [4]:
identity = np.array(
  [[[[0, 0, 0],
  [0, 1, 0],
  [0, 0, 0]]]]
)
identity.shape

(1, 1, 3, 3)

## convolvign


In [5]:
single

array([[[[5, 0, 8, 7, 8, 1],
         [1, 9, 5, 0, 7, 7],
         [6, 0, 2, 4, 6, 6],
         [9, 7, 6, 6, 8, 4],
         [8, 3, 8, 5, 1, 3],
         [7, 2, 7, 0, 1, 0]]]])

In [6]:
single.shape

(1, 1, 6, 6)

In [7]:
single[0,0 , 0:3 , 0:3 ]

array([[5, 0, 8],
       [1, 9, 5],
       [6, 0, 2]])

In [8]:
#element viase multiplication
region = single[0 , 0 , 0: 3  , 0:3] 
multiplication = region * identity
multiplication

array([[[[0, 0, 0],
         [0, 9, 0],
         [0, 0, 0]]]])

In [9]:
region @ identity

array([[[[0, 0, 0],
         [0, 9, 0],
         [0, 0, 0]]]])

In [10]:
#total is 
multiplication.sum()

9

In [11]:
newregion = single[0, 0, (0+1):(3+1) , (0+1):(3+1)]
newregion

array([[9, 5, 0],
       [0, 2, 4],
       [7, 6, 6]])

In [12]:
ans = []
single

array([[[[5, 0, 8, 7, 8, 1],
         [1, 9, 5, 0, 7, 7],
         [6, 0, 2, 4, 6, 6],
         [9, 7, 6, 6, 8, 4],
         [8, 3, 8, 5, 1, 3],
         [7, 2, 7, 0, 1, 0]]]])

In [13]:
identity

array([[[[0, 0, 0],
         [0, 1, 0],
         [0, 0, 0]]]])

In [14]:
single.shape[2]

6

In [27]:
def helper(single , filter):
    row_S = single.shape[2]
    column_S= single.shape[3]

    row_F , column_F  = filter.shape[2] , filter.shape[3]
    row_move = row_S - row_F
    col_move= column_S - column_F

    return row_move, col_move 


In [57]:
matrix = []
row_move, column_move = helper(single , identity)
#initilized ans 
ans = np.zeros((1,1,4 , 4))

for i in range(row_move + 1):
    for j in range(column_move + 1):
                    
            res = single[0, 0, (0+i):(3+i), (0+j):(3+j)]
            matrix = res * identity
            ans[0 , 0 , i , j]=  matrix.sum()


## padding

In [140]:
image = torch.as_tensor(single).float()
kernal_identity = torch.as_tensor(identity).float()


#### he functional convolution takes the kernel / 
#### filter as an argument
#### while the module has (learnable) weights to
#### represent the kernel / filter.


In [70]:
#using functional convolution 
convolved = F.conv2d(input= image, weight = kernal_identity, stride= 1 )
convolved

tensor([[[[9., 5., 0., 7.],
          [0., 2., 4., 6.],
          [7., 6., 6., 8.],
          [3., 8., 5., 1.]]]])

In [88]:
convolved = F.conv2d(input = image , weight = kernal_identity , stride = 2)
convolved

tensor([[[[9., 0.],
          [7., 6.]]]])

In [89]:
conv = nn.Conv2d(in_channels= 1, out_channels= 1, kernel_size= 3, stride= 1)
conv(image)

tensor([[[[ 1.4746e+00,  3.3917e+00, -4.9142e-01,  4.7803e-01],
          [ 2.0855e+00, -1.9216e-03, -1.2186e-01,  2.1849e+00],
          [ 9.6041e-01,  1.5007e+00,  1.6795e+00,  2.5719e+00],
          [ 1.7517e-01,  2.6866e+00,  3.4533e+00,  1.1268e+00]]]],
       grad_fn=<ConvolutionBackward0>)

## result are gibberish  now because, convolution module randomly initilizes the weights representing the kernal/filter

That’s the whole point of the convolutional module: It will learn
the kernel / filter on its own.<br>
In traditional computer vision, people would develop different<br>
filters for different purposes: blurring, sharpening, edge<br>
detection, and so on.<br><br><br>
But, instead of being clever and trying to manually devise a filter<br>
that does the trick for a given problem, why not outsource the<br>
filter definition to the neural network as well? This way, the<br>
network will come up with filters that highlight features that are<br>
relevant to the task at hand.<br><br><br>
It’s no surprise that the resulting image shows a grad_fn attribute<br>
now: It will be used to compute gradients so the network can<br>
actually learn how to change the weights representing the filter.<str>

### "Can we tell it to learn multiple filters at once?"
## Sure we can; that’s the role of the out_channels argument. If we set it to 2, 
### it will generate two (randomly initialized) filters:

In [90]:
conv_multiple = nn.Conv2d(in_channels= 1, out_channels=2, kernel_size= 3, stride=1)
conv_multiple(image)

tensor([[[[-2.0121, -3.6809, -5.8810, -6.8342],
          [-8.4024, -8.0421, -5.7923, -5.6644],
          [-8.8336, -5.7163, -5.5102, -4.7129],
          [-8.6719, -4.4343, -5.2004, -3.2441]],

         [[ 2.5142,  4.5170,  4.6755,  4.0351],
          [ 5.8265,  3.9043,  4.4481,  4.9945],
          [ 4.6511,  4.2423,  3.8007,  3.9694],
          [ 5.3306,  3.8923,  4.3593,  2.5462]]]],
       grad_fn=<ConvolutionBackward0>)

### Even if you have only one channel as input, you can have many channels as output

# -- forcing convolution module to use particular filter by setting --#

In [121]:
conv.weight[0].data 

tensor([[[-0.0159,  0.1001,  0.0744],
         [ 0.1343,  0.2389, -0.2990],
         [ 0.0747,  0.1119, -0.1390]]])

In [124]:
conv.weight[0].data

tensor([[[-0.0159,  0.1001,  0.0744],
         [ 0.1343,  0.2389, -0.2990],
         [ 0.0747,  0.1119, -0.1390]]])

In [138]:
kernal_identity

tensor([[[[5., 0., 8., 7., 8., 1.],
          [1., 9., 5., 0., 7., 7.],
          [6., 0., 2., 4., 6., 6.],
          [9., 7., 6., 6., 8., 4.],
          [8., 3., 8., 5., 1., 3.],
          [7., 2., 7., 0., 1., 0.]]]])

In [142]:
#using boring identity kernal
conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride=1, padding=0)

# Set the kernel and bias using torch.no_grad() to prevent gradient calculation
with torch.no_grad():
    conv.weight[0] = kernal_identity  # Correctly assigning the kernel weights
    conv.bias[0] = 0   # Correctly assigning the bias to 0



In [144]:
conv(image)

tensor([[[[9., 5., 0., 7.],
          [0., 2., 4., 6.],
          [7., 6., 6., 8.],
          [3., 8., 5., 1.]]]], grad_fn=<ConvolutionBackward0>)

## Padding

In [147]:
constant_padder = nn.ConstantPad2d(padding = 1 , value = 0)

In [149]:
constant_padder(image)

tensor([[[[0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 5., 0., 8., 7., 8., 1., 0.],
          [0., 1., 9., 5., 0., 7., 7., 0.],
          [0., 6., 0., 2., 4., 6., 6., 0.],
          [0., 9., 7., 6., 6., 8., 4., 0.],
          [0., 8., 3., 8., 5., 1., 3., 0.],
          [0., 7., 2., 7., 0., 1., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.]]]])

# Image Padding Documentation

This document provides details on how to use the padding arguments for image manipulation functions. Padding is used to add extra space around the edges of an image.

## Arguments

### `padding`

The `padding` argument specifies the number of columns and rows to be added around the image. It can be provided in two formats:

- **Integer**: Applies the same padding value to all sides of the image.
  
  - **Example**: `padding=2` adds 2 rows and 2 columns of padding to all sides (top, bottom, left, right).

- **Tuple**: Applies different padding values to each side of the image. The tuple is specified as `(left, right, top, bottom)`.

  - **Example**: `padding=(1, 1, 0, 0)` adds 1 column of padding to the left and right sides of the image, with no padding on the top and bottom.

### `value`

The `value` argument specifies the value that fills the newly added columns and rows. It can be any scalar value that the padding cells will be initialized with.

## Examples


In [153]:
padder1 = nn.ConstantPad2d(padding = (1,1,0,0) , value = 0)

In [154]:
padder1(image)

tensor([[[[0., 5., 0., 8., 7., 8., 1., 0.],
          [0., 1., 9., 5., 0., 7., 7., 0.],
          [0., 6., 0., 2., 4., 6., 6., 0.],
          [0., 9., 7., 6., 6., 8., 4., 0.],
          [0., 8., 3., 8., 5., 1., 3., 0.],
          [0., 7., 2., 7., 0., 1., 0., 0.]]]])

## Functional

In [156]:
padded = F.pad(input = image, pad=(1,1,1,1), mode= 'constant', value=0)
padded

tensor([[[[0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 5., 0., 8., 7., 8., 1., 0.],
          [0., 1., 9., 5., 0., 7., 7., 0.],
          [0., 6., 0., 2., 4., 6., 6., 0.],
          [0., 9., 7., 6., 6., 8., 4., 0.],
          [0., 8., 3., 8., 5., 1., 3., 0.],
          [0., 7., 2., 7., 0., 1., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.]]]])